# Google Analytics Data Formatting

In [1]:
# directories and variables
data_dir = '../data/'
google_dir = data_dir + 'ga/'

metrics_f = google_dir + 'page_path_metrics.csv'

In [2]:
import pandas as pd
pd.options.display.max_colwidth = 100

from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
metrics_df = pd.read_csv(metrics_f, skiprows=6)

# RENAME COLUMNS
adj_cols = ['year', 'country', 'page', 'new_users', 'uniq_pg_views', 'pg_views']
metrics_df.columns = adj_cols

metrics_df.head()

,year,country,page,new_users,uniq_pg_views,pg_views
0,2017,BD,/poetry-and-plays-2017,"6,956","8,211","9,909"
1,2017,IN,/poetry-and-plays-2017,"3,502","4,925","6,416"
2,2017,NP,/poetry-and-plays-2017,"1,844","2,294","2,797"
3,2017,PK,/poetry-and-plays-2017,"1,315","1,708","2,094"
4,2017,IN,/fiction-and-nonfiction-2017,"1,253","2,017","2,919"


### Pages

In [4]:
pages_df = metrics_df.page.value_counts().reset_index()
pages_df.columns = ['page', 'cnt']

pages_df.head()

,page,cnt
0,/how-writers-write-fiction-2015,306
1,/how-writers-write-fiction-2016,171
2,/poetry-and-plays-2017,167
3,/fiction-and-nonfiction-2017,156
4,/how-writers-write-fiction-2015/home,155


In [5]:
def clean_page(pg):
    # some pages start with https
    if (pg.startswith('/https://')):
        pg = pg.split('courses/')[1]
    # otherwise most start with '/'
    elif (pg.startswith('/')):
        pg = pg[1:]
    
    # remove trailing '/'
    pg_wo_slash = pg.split('/')[0]
    
    # remove trailing '#'
    pg_wo_pound = pg_wo_slash.split('#')[0]
    
    # remove trailing '?'
    pg_wo_qmark = pg_wo_pound.split('?')[0]
    
    # manual reclass
    if (pg_wo_qmark == 'flash-write-2016-credit-1'):
        pg_wo_qmark = 'flash-write-2016'
        
    return pg_wo_qmark
        
pages_df['pg_clean'] = pages_df['page'].apply(clean_page)

In [6]:
pages_df['pg_clean'].value_counts()

how-writers-write-fiction-2016    329
how-writers-write-fiction-2015    232
whitman-2016                      155
fiction-and-nonfiction-2017       102
flash-write-2016                   67
users                              51
poetry-and-plays-2017              35
                                   10
iowa-sandbox                        2
translate_c                         2
68470118-1.compliance-alex.xyz      1
www1.free-share-buttons.top         1
68470118-1.compliance-don.xyz       1
68470118-1.compliance-fred.xyz      1
home                                1
sharebutton.to                      1
Name: pg_clean, dtype: int64

### Cleaning

In [7]:
legal_vals = [
    'how-writers-write-fiction-2016',
    'how-writers-write-fiction-2015',
    'whitman-2016',
    'fiction-and-nonfiction-2017',
    'poetry-and-plays-2017',
    'flash-write-2016'
]

metrics_df['page'] = metrics_df['page'].apply(clean_page)

### Filtering

In [11]:
class_mask = (metrics_df['page'].isin(legal_vals))

print ("Before:\t{}".format(len(metrics_df)))
export_df = metrics_df[class_mask]
print ("After:\t{}\nRemoved:{}".format(len(export_df), 5000-4716))

export_df.head()

Before:	5000
After:	4716
Removed:284


,year,country,page,new_users,uniq_pg_views,pg_views
0,2017,BD,poetry-and-plays-2017,"6,956","8,211","9,909"
1,2017,IN,poetry-and-plays-2017,"3,502","4,925","6,416"
2,2017,NP,poetry-and-plays-2017,"1,844","2,294","2,797"
3,2017,PK,poetry-and-plays-2017,"1,315","1,708","2,094"
4,2017,IN,fiction-and-nonfiction-2017,"1,253","2,017","2,919"


### Rolling/Summing Up

In [27]:
test_mask = (export_df['year'] == 2015) & (export_df['country'] == 'AE')

export_df[test_mask]

,year,country,page,new_users,uniq_pg_views,pg_views
543,2015,AE,how-writers-write-fiction-2015,17,102,249
645,2015,AE,how-writers-write-fiction-2015,14,21,24
974,2015,AE,how-writers-write-fiction-2015,7,77,97
1498,2015,AE,how-writers-write-fiction-2015,4,4,4
1499,2015,AE,how-writers-write-fiction-2015,4,4,4
1939,2015,AE,how-writers-write-fiction-2015,2,2,2
1940,2015,AE,how-writers-write-fiction-2015,2,21,35
2827,2015,AE,how-writers-write-fiction-2015,1,1,1
2828,2015,AE,how-writers-write-fiction-2015,1,6,9
2829,2015,AE,how-writers-write-fiction-2015,1,9,10


In [37]:
export_df.to_csv('unrolled.csv', index=False)

In [40]:
export_df.groupby(['year', 'country', 'page']).sum()

new_users  \
year country page                                                                                                                   
2015 AE      how-writers-write-fiction-2015                                                                  17147442211111111111   
     AF      how-writers-write-fiction-2015                                                                                   111   
     AG      how-writers-write-fiction-2015                                                                                 42111   
     AL      how-writers-write-fiction-2015                                                                            2111111111   
     AM      how-writers-write-fiction-2015                                                                                     1   
     AO      how-writers-write-fiction-2015                                                                                     1   
     AR      how-writers-write-fiction-2015                                                                       212014742211111   
     AS      how-writers-write-fiction-2015                                                                                   211   
     AT      how-writers-write-fiction-2015                                                                           74222211111   
     AU      how-writers-write-fiction-2015                     20018678554034221514121212106544442222222221111111111111111111111   
     AW      how-writers-write-fiction-2015                                                                                     2   
     AZ      how-writers-write-fiction-2015                                                                                     1   
     BA      how-writers-write-fiction-2015                                                                                  2111   
     BB      how-writers-write-fiction-2015                                                                                  1941   
     BD      how-writers-write-fiction-2015                                                                             126422111   
     BE      how-writers-write-fiction-2015                                                                          126541111111   
     BG      how-writers-write-fiction-2015                                                                                 22211   
     BH      how-writers-write-fiction-2015                                                                              21111111   
     BI      how-writers-write-fiction-2015                                                                                     1   
     BJ      how-writers-write-fiction-2015                                                                                     1   
     BM      how-writers-write-fiction-2015                                                                                   611   
     BO      how-writers-write-fiction-2015                                                                               2111111   
     BR      how-writers-write-fiction-2015                                                    1421145221997655544222111111111111   
     BS      how-writers-write-fiction-2015                                                                                422111   
     BT      how-writers-write-fiction-2015                                                                                     2   
     BW      how-writers-write-fiction-2015                                                                                 41111   
     BY      how-writers-write-fiction-2015                                                                                   111   
     BZ      how-writers-write-fiction-2015                                                                                     1   
     CA      how-writers-write-fiction-2015  635267108753931201919191611109765444444422222222211111111111111111111111111111111111 

### Final

Final file is located at `/data/canonical/mooc-metrics.csv`. I rolled the rows using Excel because Pandas was having sum issues with summation.